### Assignment 1 - Invasive Species

### Setup

In [4]:
%matplotlib inline

In [2]:
path = 'data/invasivespecies/'
# path = 'data/invasivespecies/sample/

In [3]:
from __future__ import division, print_function

import os, json
from glob import glob
import pandas as pd
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

import utils; reload(utils)
from utils import plots

# Dataset formatting
from os import walk

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
Using Theano backend.


### Create Validation set and sample

In [1]:
# Just testing out command line
%cd data
%cd ../

/home/ubuntu/courses/deeplearning1/nbs/data
/home/ubuntu/courses/deeplearning1/nbs


In [3]:
!kg config -g -c 'invasive-species-monitoring'

In [16]:
# Unzip training folder
originals_folder = path+'train_original/'
# !7z --help
if not os.path.exists(originals_folder):
    !7z e {path}train.7z -o{output_folder}

In [40]:
# Get training labels
training_labels_df = pd.read_csv(path + 'train_labels.csv')
size = training_labels_df.size
num_positives = training_labels_df['invasive'].sum()
print('Size:', size)
print('Positives: {} Percent: {}'.format(num_positives, num_positives/size))


Size: 4590
Positives: 1448 Percent: 0.315468409586


In [41]:
# Get training files
import re

_, _, filenames = next(walk(originals_folder))


p = re.compile('^([0-9]+).jpg')
def find_file_id(filename):
    m = p.match(filename)
    if m is not None:
        return int(m.group(1))
    else:
        print('Could not regex filename: ', filename)
        return -1
file_ids = map(find_file_id, filenames)

# Merge data into one dataframe:
file_df = pd.DataFrame([file_ids, filenames], index=['name', 'file']).transpose()

In [50]:
# Label files and move to labeled folder
from shutil import copyfile

labeled_folder = path+'labeled/'
if not os.path.exists(labeled_folder):
    os.makedirs(labeled_folder)
    
labeled_df = pd.merge(training_labels_df, file_df, on='name')
for row in labeled_df.itertuples():
    label = 'invasive.' if row[2] == 1 else 'noninvasive.'
    file_name = row[3]
    new_file = labeled_folder+label+file_name;
    copyfile(originals_folder+file_name, new_file)
    

In [73]:
import shutil
train_folder = path+'train/'
shutil.copytree(labeled_folder, train_folder)
_, _, filenames = next(walk(train_folder))
shuf = np.random.permutation(filenames)
size = len(filenames)
print(size)

valid_folder = path+'valid/'
if not os.path.exists(valid_folder):
    os.makedirs(valid_folder)
    
sample_train_folder = path+'sample/train/'
if not os.path.exists(sample_train_folder):
    os.makedirs(sample_train_folder)
for i in range(200):
    copyfile(train_folder+shuf[i], sample_train_folder+shuf[i])
    
sample_valid_folder = path+'sample/valid/'
if not os.path.exists(sample_valid_folder):
    os.makedirs(sample_valid_folder)
for i in range(200):
    copyfile(train_folder+shuf[i], sample_valid_folder+shuf[i])
    
validation_size = int(round(size * .3))
for i in range(validation_size):
    os.rename(train_folder+shuf[i], valid_folder+shuf[i])



2295


In [95]:
def classify_folders(folder):
    if not os.path.exists(folder+'noninvasive'):
        os.makedirs(folder+'noninvasive')
        os.makedirs(folder+'invasive')
    inv = glob(folder+'invasive.*.jpg')
    noninv = glob(folder+'noninvasive.*.jpg')
    move_files_to(inv, folder, folder+'invasive/')
    move_files_to(noninv, folder, folder+'noninvasive/')
    
def move_files_to(files, old_folder, new_folder):
    for fname in files:
        newf = fname.replace(old_folder, new_folder)
        os.rename(fname, newf)
        
classify_folders(sample_train_folder)

In [91]:
inv = glob(sample_train_folder+'invasive.*.jpg')
f = inv[0]
f
newf = f.replace(sample_train_folder, sample_train_folder+'invasive/')
newf

'data/invasivespecies/sample/train/invasive/invasive.1903.jpg'

# VGG

In [ ]:
# batch_size=64
batch_size=8

In [ ]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [ ]:
vgg = Vgg16()

#### Visualize data

In [ ]:
batches = vgg.get_batches(path+'train', batch_size=4)
imgs,labels = next(batches)

In [ ]:
print(imgs.shape)
print(labels.shape)

In [ ]:
imgs[0, :, 0, 0]

In [ ]:
plots(imgs, titles=labels)

In [ ]:
vgg.predict(imgs, True)

In [ ]:
vgg.classes[:4]

### Finetune Cats and Dogs

In [4]:
def train_vgg():
    batch_size=64
    batches = vgg.get_batches(path+'train', batch_size=batch_size)
    val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2)
    vgg.finetune(batches)
    vgg.fit(batches, val_batches, nb_epoch=1)

    vgg.model.save_weights(weights_fullpath)

In [ ]:
from keras.utils.data_utils import get_file
weights_file = 'trained_weights.h5'
weights_fullpath = path+'models/'+weights_file
if os.path.exists(weights_fullpath):
    vgg.model.load_weights(weights_fullpath)
    print('Loaded model from cache')
else:
    train_vgg()

### Predict results in test set

In [ ]:
folderName = 'test'
num_test_images = 12500
test_batch_size = 100

# folderName = 'sample/test'
# num_test_images = 8
# test_batch_size = 1

batches = vgg.get_batches(path+folderName, batch_size=test_batch_size, shuffle=False, class_mode=None)
# with open('train.csv', 'wb') as csvfile:


Let's look at our predictions...

Need to convert file names to label ids

In [ ]:
import re
filenames = batches.filenames

p = re.compile('.*/([0-9]+).jpg')
def find_file_id(filename):
    m = p.match(filename)
    if m is not None:
        return int(m.group(1))
    else:
        print('Could not regex filename: ', filename)
        return -1
file_ids = map(find_file_id, filenames)

Option A: Predict values using vgg test function (Not recommended. No progress)

Option B: Predict values with batches and progress

In [ ]:
import math
from tqdm import tqdm

predict_file = path + 'models/predict.npy'

def predict_test():
    batches = vgg.get_batches(path+folderName, batch_size=test_batch_size, shuffle=False, class_mode=None)

    p_results = np.zeros(num_test_images)
    current_index = 0
    # Iterative loop
    for batch in tqdm(batches, total=math.ceil(num_test_images/test_batch_size)):
        if batch is None:
            break
        p = vgg.model.predict_on_batch(batch)
        p_dog = p[:, 1]
        p_size = p.shape[0]
    #     print('Predictions: {}\n Size: {}'.format(dog_p, p_size))
        new_index = current_index + p_size
        p_results[current_index:new_index] = p_dog
        current_index = new_index
        if current_index >= num_test_images:
            break
    print(p_results)
    np.save(predict_file)

if os.path.exists(predict_file):
    np.load(predict_file)
    print('Loaded predictions from cache')
else:
    predict_test()

In [ ]:
# Verify the arrays match
print(p_results.shape)
print(len(file_ids))

### Combine ids with labels and save

In [ ]:
import pandas as pd

agg = pd.DataFrame({'id': file_ids, 'label': p_results})
agg = agg.sort_values(['id'])
print(agg)
agg.to_csv(path + 'predictions.csv', index=False)


------

## Implementing VGG with Keras Backend

In [ ]:
from numpy.random import random, permutation
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image

In [ ]:
FILES_PATH = 'http://files.fast.ai/models/'
CLASS_FILE='imagenet_class_index.json'
fpath = get_file(CLASS_FILE, FILES_PATH+CLASS_FILE, cache_subdir='models')
with open(fpath) as f:
    class_dict = json.load(f)
# classes = [class_dict[i][1] for i in class_dict]
classes = [class_dict[str(i)][1] for i in range(len(class_dict))] # original

## Model creation

In [ ]:
def ConvBlock(layers, model, filters):
    for i in range(layers): 
        model.add(ZeroPadding2D((1,1)))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

In [ ]:
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

In [ ]:
# ??Convolution2D

# ??MaxPooling2D

# ??Dense

In [ ]:
# Mean of each channel as provided by VGG researchers
vgg_mean = np.array([123.68, 116.779, 103.939]).reshape((3,1,1))

def vgg_preprocess(x):
    x = x - vgg_mean     # subtract mean
    return x[:, ::-1]    # reverse axis bgr->rgb

In [ ]:
def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,224,224)))

    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)

    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    return model

In [ ]:
??MaxPooling2D

??Dense

In [ ]:
model = VGG_16()

In [ ]:
fpath = get_file('vgg16.h5', FILES_PATH+'vgg16.h5', cache_subdir='models')
model.load_weights(fpath)

In [ ]:
batch_size=4

In [ ]:
def get_batches(dirname, gen=image.ImageDataGenerator(), shuffle=True, 
                batch_size=batch_size, class_mode='categorical'):
    return gen.flow_from_directory(path+dirname, target_size=(224,224), 
                class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

In [ ]:
batches = get_batches('train', batch_size=batch_size)
val_batches = get_batches('valid', batch_size=batch_size)
imgs,labels = next(batches)

# This shows the 'ground truth'
plots(imgs, titles=labels)

In [ ]:
def pred_batch(imgs):
    preds = model.predict(imgs)
    idxs = np.argmax(preds, axis=1)

    print('Shape: {}'.format(preds.shape))
    print('First 5 classes: {}'.format(classes[:5]))
    print('First 5 probabilities: {}\n'.format(preds[0, :5]))
    print('Predictions prob/class: ')
    
    for i in range(len(idxs)):
        idx = idxs[i]
        print ('  {:.4f}/{}'.format(preds[i, idx], classes[idx]))

In [ ]:
pred_batch(imgs)